# TP $k$ Nearest Neighbors

Save the notebook as either PDF or HTML and make sure all the results are saved correctly (I won't run them and the original format does not save the results automatically), **and put your name in the filename**.

<div class="alert alert-success"> 
    <b>Questions are in green boxes.</b>
The maximum time you should spend on each question is given as indication only. If you take more time than that, then you should come see me.
</div>
<div class="alert alert-info" role="alert"><b>Analyzes are in blue boxes.</b> You should comment on your results in theses boxes (Is it good? Is it expected? Why do we get such result? Why is it different from the previous one? etc)
</div>

In [ ]:
import jax
import jax.numpy as jnp
import jax.nn as jnn
import numpy as np
import matplotlib.pyplot as plt

For this lab, we will use the [MNIST dataset (~15Mo)](https://en.wikipedia.org/wiki/MNIST_database). It consists of 28x28 images (loaded as a 784 vector) and the associated label for training, validation and test sets. 

The following code loads a subset of 1000 training samples and 500 validation samples.

In [ ]:
# Load the dataset
data = np.load('mnist1k.npz')
X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
N_train = len(y_train)
N_val = len(y_val)
plt.imshow(X_train[0,:].reshape(28,28))
print(y_train[0])

Next, we define the 0-1 loss that measures the error rate of a classifier.

In [ ]:
def error_rate(y_hat, y):
    return (1.-(y_hat==y)).mean()

## Implementing a simple $k$-NN


<div class="alert alert-success"> <b> Q1.</b> 
Implement the code of the $k$-NN classifier in the following class. It has to work for any value of $k$ and any number of classes in $\mathcal{Y}$. Make sure it does not have any explicit loop and that it runs instantaneously. We first try a simple 1-NN on the full training set to check that our code works. I found around 40% error (compared to 90% for random guesses). <i>(Indicative time: maximum 45 minutes)</i>
</div>


In [ ]:
class KNearestNeighbor():
    def __init__(self, X, y, k=1):
        self.X = X
        self.y = jnn.one_hot(y, 10)
        self.k = k
    
    '''
    x is a matrix nxd of n samples of dimension d
    returns a vector of size n containing the prediction of the class
    '''
    def __call__(self, x):
        ## TODO
        return 

In [ ]:
nn = KNearestNeighbor(X_train, y_train, k=1)
y_hat = nn(X_val)
err = error_rate(y_hat, y_val)
print(err)

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

## selecting $k$ using cross-validation

Next, we want to select the optimal value for $k$ using K-Fold cross-validation.

<div class="alert alert-success"> <b>Q2. </b> 
Code a routine that performs 10-Fold cross-validation, use it to plot the average and stddev error rate  of the $k$-NN classifier for $k$ varying from 1 to 32 by power of 2. Make sure it runs quasi-instantaneously for each $k$ and that it only has one explicit loop in the routine (the one over the splits) and only one outside (the one over $k$). I found around 8 to perform best <i>(indicative time: maximum 45 minutes)</i>
</div>

In [ ]:
def tenfold(cls, X_train, y_train, k):
    ## TODO
    return 

##TODO
for k in [1, 2, 4, 8, 16, 32]:
    ## TODO

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>  Answer
</div>

## PCA preprocessing

The euclidean distance may not be the best to measure the distance between images. In this particular example, we know that the borders are mostly 0, and thus they introduce 0 in the computation that tend to make all images more similar.

To aleviate this problem, we propose to project the data into a lower dimensional space using Principal Component analysis.

<div class="alert alert-success"> <b>Q3.</b> 
    Code a class that performs PCA. <i>(indicative time: maximum 20 minutes) </i>
</div>

In [ ]:
class PCA():
    '''
    dim is the number of dimensions retained by the projection.
    '''
    def __init__(self, dim):
        self.d = dim
        self.mean = None
        self.P = None
    '''
    Computes and stores self.mean and self.P from the training data X
    '''
    def train(self, X):
        ## TODO
    '''
    Projects X using self.mean and self.P
    '''
    def transform(self, X):
        return jnp.matmul(X - self.mean[None, :], self.P)

<div class="alert alert-success"> <b>Q4.</b> 
Train a 1-NN on the full training set pre-processed using a PCA that keeps only 64 most relevant dimensions, and computes the error rate. Compare to the original result and analyze. This should be quasi-instantaneous. I found PCA works much better. <i>(indicative time: maximum 10 minutes)</i>
</div>

In [ ]:
## TODO

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>
 Answer
</div>

<div class="alert alert-success"> <b>Q5.</b> 
Perform a 10-Fold cross-validation search over k from 1 to 20 and PCA dimensions of increasing power of 2 (*e.g.*, 2, 4, 8, ...), and analyze your findings. I found good scores for low values of $k$ and large values of PCA. <i>(indicative time: maximum 45 minutes)</i>
</div>

In [ ]:
## TODO

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>
Answer
</div>

## Weighted $k$-NN

In the classical $k$-NN, all neighbors vote with the same weight towards the predicted class. However, one could argue that this doe not take into account how far these neighbors are. For example, if the example $x$ we want to classify is in a low density region, the value of $k$ may be to high, and the vote is cast by training samples that are very far away from $x$.

To aleviate this problem, we propose to weight the vote cast by the neighbor by the inverse proportion of their distance. Let us denote $x_1, \dots, x_k$ the $k$ nearest neighbors of $x$ in increasing distance order, and let us denote by $d_i = \|x_i - x\|^2$ the squared distance between $x$ and $x_i$. Then, the weight is simply
$$ w_i = \frac{\sum_m d_m}{d_i} $$

<div class="alert alert-warning"> <b>Q6 [Optional].</b> 
Implement the weighted $k$-NN class.
</div>

In [ ]:
class KNearestNeighbor():
    def __init__(self, X, y, k=1):
        self.X = X
        self.y = y
        self.k = k
    
    '''
    x is a matrix nxd of n samples of dimension d
    returns a vector of size n containing the prediction of the class
    '''
    def __call__(self, x):
        return

<div class="alert alert-warning"> <b>Q7 [Optional].</b> 
Perform the same 10-Fold cross-validation search over $k$ and the PCA dimension for the weighted $k$-NN classifier and analyze your results.
</div>

<div class="alert alert-info" role="alert"><b>Analyze your results in this box.</b>
Answer
</div>